In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import torch.nn.functional as F

In [2]:
dtype = torch.FloatTensor
dtype

torch.FloatTensor

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

#### 定义数据，以及一些常规参数

In [11]:
sentences = ["i love you", "he loves me", "she likes baseball", "i hate you", "sorry for that", "this is awful"]

In [12]:
sentences

['i love you',
 'he loves me',
 'she likes baseball',
 'i hate you',
 'sorry for that',
 'this is awful']

In [5]:
labels = [1, 1, 1, 0, 0, 0]  # 1 is good, 0 is not good.
labels

[1, 1, 1, 0, 0, 0]

In [6]:
# TextCNN Parameter
embedding_size = 2
embedding_size

2

In [16]:
sequence_length = 3
sequence_length

3

In [17]:
num_classes = len(set(labels))
num_classes

2

In [18]:
batch_size = 3

In [19]:
word_list = ' '.join(sentences).split()
word_list

['i',
 'love',
 'you',
 'he',
 'loves',
 'me',
 'she',
 'likes',
 'baseball',
 'i',
 'hate',
 'you',
 'sorry',
 'for',
 'that',
 'this',
 'is',
 'awful']

In [20]:
vocab = list(set(word_list))
vocab

['awful',
 'sorry',
 'love',
 'loves',
 'that',
 'me',
 'i',
 'he',
 'she',
 'this',
 'you',
 'is',
 'baseball',
 'hate',
 'for',
 'likes']

In [21]:
word2idx = {w:i for i, w in enumerate(vocab)}
word2idx

{'awful': 0,
 'sorry': 1,
 'love': 2,
 'loves': 3,
 'that': 4,
 'me': 5,
 'i': 6,
 'he': 7,
 'she': 8,
 'this': 9,
 'you': 10,
 'is': 11,
 'baseball': 12,
 'hate': 13,
 'for': 14,
 'likes': 15}

In [22]:
vocab_size = len(vocab)
vocab_size

16

#### 数据预处理

In [27]:
def make_data(sentences, labels):
    inputs = []
    for sen in sentences:
        #print(sen)
        inputs.append([word2idx[n] for n in sen.split()])
    #print(inputs)
    targets = []
    for out in labels:
        targets.append(out)
    return inputs, targets
input_batch, target_batch = make_data(sentences, labels)

In [28]:
input_batch

[[6, 2, 10], [7, 3, 5], [8, 15, 12], [6, 13, 10], [1, 14, 4], [9, 11, 0]]

In [29]:
target_batch

[1, 1, 1, 0, 0, 0]

In [30]:
input_batch, target_batch = torch.LongTensor(input_batch), torch.LongTensor(target_batch)

In [31]:
input_batch

tensor([[ 6,  2, 10],
        [ 7,  3,  5],
        [ 8, 15, 12],
        [ 6, 13, 10],
        [ 1, 14,  4],
        [ 9, 11,  0]])

In [32]:
target_batch

tensor([1, 1, 1, 0, 0, 0])

In [33]:
dataset = Data.TensorDataset(input_batch, target_batch)
dataset

In [34]:
loader = Data.DataLoader(dataset, batch_size, True)
loader

#### 构建模型

In [35]:
class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()
        self.w = nn.Embedding(vocab_size, embedding_size)
        output_channel = 3
        self.conv = nn.Sequential(
            # (2, embedding_size)为卷积核的大小，（filter_height, filter_width）
            nn.Conv2d(1, 3, (2, embedding_size)),
            nn.ReLU(),
            # （filter_height, filter_width）
            nn.MaxPool2d((2, 1)),
        )
        self.fc = nn.Linear(output_channel, num_classes)
        
    def forward(self, x):
        batch_size = x.shape[0]
        embedding_x = self.w(x) # [batch_size, sequence_length, embedding_size]
        embedding_x = embedding_x.unsqueeze(1) # # add channel(=1) [batch, channel(=1), sequence_length, embedding_size]
        conved = self.conv(embedding_x)
        flatten = conved.view(batch_size, -1)
        output = self.fc(flatten)
        return output

In [36]:
model = TextCNN().to(device)
model

TextCNN(
  (w): Embedding(16, 2)
  (conv): Sequential(
    (0): Conv2d(1, 3, kernel_size=(2, 2), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3, out_features=2, bias=True)
)

In [37]:
criterion = nn.CrossEntropyLoss().to(device)
criterion

CrossEntropyLoss()

In [38]:
opitimizer = optim.Adam(model.parameters(), lr = 0.01)
opitimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)

In [41]:
epoch_list = []
loss_list = []
for epoch in range(5000):
    epoch_list.append(epoch)
    for batch_x, batch_y in loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        pred = model(batch_x)
        loss = criterion(pred, batch_y)
        loss_list.append(loss)
        if (epoch + 1) % 1000 == 0:
            print('Epcoh:', '%4d'%(epoch + 1), 'loss = ','{:.6f}'.format(loss))
        
        opitimizer.zero_grad()
        loss.backward()
        opitimizer.step()

Epcoh: 1000 loss =  0.000006
Epcoh: 1000 loss =  0.000003
Epcoh: 2000 loss =  0.000001
Epcoh: 2000 loss =  0.000001
Epcoh: 3000 loss =  0.000000
Epcoh: 3000 loss =  0.000000
Epcoh: 4000 loss =  0.000000
Epcoh: 4000 loss =  0.000000
Epcoh: 5000 loss =  0.000000
Epcoh: 5000 loss =  0.000000


In [43]:
len(epoch_list)

5000

#### 测试

In [54]:
test_text = 'i hate me'
tests = [[word2idx[n] for n in test_text.split()]]
test_batch = torch.LongTensor(tests).to(device)

In [55]:
tests

[[6, 13, 5]]

In [56]:
test_batch

tensor([[ 6, 13,  5]])

In [57]:
model = model.eval()

In [58]:
model

TextCNN(
  (w): Embedding(16, 2)
  (conv): Sequential(
    (0): Conv2d(1, 3, kernel_size=(2, 2), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3, out_features=2, bias=True)
)

In [59]:
predict = model(test_batch).data.max(1, keepdim = True)[1]
if predict[0][0] == 0:
    print(test_text, 'is a Bad mean')
else:
    print(test_text, 'is a good mean')

i hate me is a Bad mean
